In [ ]:
import time
import struct 

import numpy as np
import matplotlib.pyplot as plt

import os

from ymaeda_tools.read_snapshot import read_snapshot_params, snapshot_loc2idx, snapshot_XYZ, extract_snapshot

# This notebook performs checks on the creation of the Green's functions associated with a specific
# source location, observation location and direction of motion.

df = 0.002441406 # frequency step size used by YMAEDA_TOOLS
f = np.arange(0, df * 2049, df) # frequency half space
F = np.arange(0, df * 4096, df) # frequency full space

dx = 10
dy = 10
dz = 10

# location to create the inversion Green's function:
X = -10900
Y = -121100
Z = 1000
ooo = np.array([X, Y, Z])

# However, we do not need the Green's function at the coordinates above. In order to create the inversion
# Green's functions, we need to load the Green's functions at the 8 cells surrounding the point of interest.
mmm = np.array([X - dx, Y - dy, Z - dz])
mmp = np.array([X - dx, Y - dy, Z + dz])
mpm = np.array([X - dx, Y + dy, Z - dz])
mpp = np.array([X - dx, Y + dy, Z + dz])
pmm = np.array([X + dx, Y - dy, Z - dz])
pmp = np.array([X + dx, Y - dy, Z + dz])
ppm = np.array([X + dx, Y + dy, Z - dz])
ppp = np.array([X + dx, Y + dy, Z + dz])

#HDD_LOC = "/Users/ynatsume/jupyter_main/Shinmoedake Data/" 
HDD_LOC = "/Volumes/TOSHIBA_HDD/Final Year Projects/Shinmoedake Data/EIC_BACKUP/home/natsume/"

In [ ]:
STATIONS = ["SMN_EW_SMALL", "SMN_NS_SMALL", "SMN_UD_SMALL"]

GREENS_FUNCTION = "pow34"

t = []
g = []

for i, STATION in enumerate(STATIONS):
    if GREENS_FUNCTION == 'pow5':
        snapshot_dir = HDD_LOC + "GFpow5/" + STATION + "/PML/snapshot/"
    elif GREENS_FUNCTION == 'pow34':
        snapshot_dir = HDD_LOC + "GFpow34/" + STATION + "/PML/snapshot/"
    elif GREENS_FUNCTION == 'cos':
        snapshot_dir = HDD_LOC + "GFcos/" + STATION + "/PML/snapshot/"
    
    # First of all we need to load the 3 components of the Green's function measured at the wanted location...
    start_time = time.time()
    t_tmp, g_tmp, N, x0, dx = extract_snapshot(snapshot_dir, ooo[0], ooo[1], ooo[2], return_params = True)
    print("{:.2f}s".format(time.time() - start_time))
    
    t.append(t_tmp)
    g.append(g_tmp)

t = np.array(t)
g = np.array(g)
dx = np.array(dx)
N = np.array(N)
x0 = np.array(x0)

In [ ]:
print("np.shape(t)", np.shape(t))
print("np.shape(g)", np.shape(g))
print("N", N)
print("x0", x0)
print("dx", dx)

In [ ]:
plt.figure(figsize = (15, 6))
plt.subplot(3, 1, 1)
plt.plot(t[0], g[0, :, 0], t[0], g[0, :, 1], t[0], g[0, :, 2]) # 3D GF due to SMN_EW force
plt.legend(["x", "y", "z"])
plt.ylabel("{} (m/s)".format(STATIONS[0]))
plt.grid(True)
plt.subplot(3, 1, 2)
plt.plot(t[1], g[1, :, 0], t[1], g[1, :, 1], t[1], g[1, :, 2]) # 3D GF due to SMN_NS force
plt.legend(["x", "y", "z"])
plt.ylabel("{} (m/s)".format(STATIONS[1]))
plt.grid(True)
plt.subplot(3, 1, 3)
plt.plot(t[2], g[2, :, 0], t[2], g[2, :, 1], t[2], g[2, :, 2]) # 3D GF due to SMN_UD force
plt.legend(["x", "y", "z"])
plt.ylabel("{} (m/s)".format(STATIONS[2]))
plt.grid(True)
plt.xlabel("Time (s)")
plt.show()

In [ ]:
# To-do: set up the following matrix:
#       d G_0x  d G_0y  d G_0z  d G_0x  d G_0y  d G_0z    
# G_0 = ------, ------, ------, ------, ------, ------.
#        d x     d y     d z     d y     d z     d x
#       d G_1x  d G_1y  d G_1z  d G_1x  d G_1y  d G_1z    
# G_1 = ------, ------, ------, ------, ------, ------.
#        d x     d y     d z     d y     d z     d x
#       d G_2x  d G_2y  d G_2z  d G_2x  d G_2y  d G_2z    
# G_2 = ------, ------, ------, ------, ------, ------.
#        d x     d y     d z     d y     d z     d x

# g has the structure: [Force direction x Time steps x Ground displacement direction]
# e.g. [3 x 121 x 3]

# To perform the spatial derivatives, the Green's functions values at the neighbouring grid points
# must also be measured. For the 1st order central difference method, the grid points to the left
# and right must be measured as well, and the derivative taken by: f'(x) = (f(x+h) - f(x-h))/2h.
# Therefore, for each grid point, there are 8 other grid points of data to load for a total of 9!

G = np.zeros([len(t[0]), 3, len(STATIONS)])

for n in range(len(t[0])):
    G[n] = g[:, n, :].T # Note that: G[:, :, 0] == g[0, :, :]
    
# G has the structure: [Time steps x Ground displacement direction x Force direction]
# e.g. [121 x 3 x 3]
print("np.shape(G):",np.shape(G)) 

plt.figure(figsize = (15, 3))
plt.plot(t[0], g[0, :, :]) # Note that: G[:, :, 0] == g[0, :, :] if the transformation is performed correctly
plt.plot(t[0], G[:, :, 0], ".") 
plt.ylabel("{} (m/s)".format(STATIONS[0]))
plt.xlabel("Time (s)")
plt.grid(True)
plt.xlim([0.5, 3.0])
plt.show()